In [16]:
import numpy as np
import pandas as pd
from scipy.stats import poisson
import seaborn as sns
import matplotlib.pyplot as plt


In [17]:
data = pd.read_csv("em_data.txt", header=None)
children_count = data[0].values 

In [ ]:
unique_values, frequencies = np.unique(children_count, return_counts=True)
# Plot frequency plot
plt.figure(figsize=(10, 6))
plt.bar(unique_values, frequencies, color='skyblue', edgecolor='black')
plt.xlabel('Number of Children', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('Frequency Plot of Children Count', fontsize=16)
plt.xticks(unique_values, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig("Unique_Counts.png")
plt.show()

In [19]:
def em_algorithm(data, max_iter=100, tol=1e-6):
	# Initialization of the parameters for the algo
	n = len(data)
	print("Mean of the data: ", np.mean(data))
	lambda_1 = np.mean(data) * 0.8  # Initial mean for families with family planning
	lambda_2 = np.mean(data) * 1.2 # Initial mean for families without family planning
	pi_1 = 0.5  # Initial proportion of families with family planning
	pi_2 = 0.5  # Initial proportion of families without family planning

	prev_log_likelihood = -np.inf

	for iteration in range(max_iter):
		# E-step: Calculate Posterior Probabilities
		responsibility_1 = pi_1 * poisson.pmf(data, lambda_1)
		responsibility_2 = pi_2 * poisson.pmf(data, lambda_2)

		total_responsibility = responsibility_1 + responsibility_2

		responsibility_1 /= total_responsibility
		responsibility_2 /= total_responsibility

		# M-step: Update parameters
		# Update lambda_1 and lambda_2 (mean number of children for each group)
		lambda_1 = np.sum(responsibility_1 * data) / np.sum(responsibility_1)
		lambda_2 = np.sum(responsibility_2 * data) / np.sum(responsibility_2)

		# Update pi_1 and pi_2 
		pi_1 = np.mean(responsibility_1)
		pi_2 = 1- pi_1

		log_likelihood = np.sum(np.log(total_responsibility))

		if np.abs(log_likelihood - prev_log_likelihood) < tol:
			print(f"Converged after {iteration+1} iterations.")
			print(f"Iteration {iteration+1}: lambda_1={lambda_1}, lambda_2={lambda_2}, pi_1={pi_1}, pi_2={pi_2}, log-likelihood={log_likelihood}")
			break
		prev_log_likelihood = log_likelihood
		print(f"Iteration {iteration+1}: lambda_1={lambda_1}, lambda_2={lambda_2}, pi_1={pi_1}, pi_2={pi_2}, log-likelihood={log_likelihood}")

	return lambda_1, lambda_2, pi_1, pi_2


		

In [ ]:
lambda_1, lambda_2, pi_1, pi_2 = em_algorithm(children_count, max_iter = 200)

print(f"Estimated lambda_1 (mean children with family planning): {lambda_1}")
print(f"Estimated lambda_2 (mean children without family planning): {lambda_2}")
print(f"Estimated pi_1 (proportion with family planning): {pi_1}")
print(f"Estimated pi_2 (proportion without family planning): {pi_2}")

In [21]:
def plot_poisson_densities(data, lambda_1, lambda_2,pi_1, pi_2):
    import seaborn as sns
    import matplotlib.pyplot as plt
    x = np.arange(0, max(data) + 1)
    poisson_lambda_1 = pi_1 * poisson.pmf(x, lambda_1)
    poisson_lambda_2 = pi_2 * poisson.pmf(x, lambda_2)
    poisson_mixture = poisson_lambda_1 + poisson_lambda_2

   
    print(poisson_mixture)
    sns.histplot(data, bins=25, kde=False, stat="density", color="blue", edgecolor="black", label="Data Histogram")
    plt.plot(x, poisson_mixture, label="Poisson Mixture Model", color="green", linewidth=2, linestyle="--")

    plt.xlabel("Number of Children")
    plt.ylabel("Density Value")
    plt.title("Density Plot and Fitted Poisson Mixture Model")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
plot_poisson_densities(children_count, lambda_1, lambda_2,pi_1, pi_2)
